<a href="https://colab.research.google.com/github/rs5434/QIC/blob/main/ibm_qiskit_image_processing_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#!pip install qiskit qiskit-aer

In [ ]:
import numpy as np
from numpy import pi
import cv2
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt

In [ ]:
def rotations(quantum_circuit, n):  # Installs necessary rotation gates onto an empty circuit
    if n == 0:  # Returns finalised generated circuit upon meeting the last qubit
        return quantum_circuit
    n -= 1  # Decrements current qubit to continue generating circuit
    """Generates gates for QFT"""
    quantum_circuit.h(n)  # Hadamard gate application
    for qubit in range(n):
        quantum_circuit.cp(pi/2**(n-qubit), qubit, n)  # Controlled-phase gate application
    return rotations(quantum_circuit, n)  # Utilises recursion to rotate the rest of the n-1 qubits

In [ ]:
def swap(quantum_circuit, n):  # Swaps qubits in order to match QFT definition
    for qubit in range(n//2):
        quantum_circuit.swap(qubit, n-qubit-1)
    return quantum_circuit

In [ ]:
def qft(quantum_circuit, n):  # Generalised QFT-circuit generator based on mathematical derivation
    """Generation is split into two functions"""
    rotations(quantum_circuit, n)
    swap(quantum_circuit, n)
    return quantum_circuit  # Returns finalised circuit

In [ ]:
def image_to_binary(image):  # Converts image to binary and presents data in a flattened NumPy array
    image_array = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]
    return image_array.flatten(order='C')  # Returns flattened image pixel values in a row-wise manner

In [ ]:
def basis_encode(array):  # Encodes array into a quantum circuit via computational basis states
    quantum_circuit = QuantumCircuit(array.size)  # Creates a new circuit with enough qubits to hold the image
    n = 0  # Begins encoding from 0th qubit
    for bit in array:
        if bit == 255:  # Applies Pauli-X gate to qubit to represent coloured value
            quantum_circuit.x(n)
        elif bit == 0:
            pass
        n += 1  # Continues to encode the next qubit
    return quantum_circuit  # Returns the finalised quantum circuit

In [ ]:
def main():
    # Download the image using wget
    !wget https://github.com/ica574/quantum-image-processing/raw/main/test.png
    # Load the downloaded image using OpenCV
    image = cv2.imread("test.png", cv2.IMREAD_GRAYSCALE)
    circuit = basis_encode(image_to_binary(image))  # Encodes the image onto a quantum circuit
    simulator = AerSimulator()  # Qiskit Aer simulator instantiation

    """Computational basis qubits"""
    qc_init = circuit.copy()
    qc_init.save_statevector()
    result = simulator.run(qc_init).result()
    statevector = result.get_statevector()
    plot_bloch_multivector(statevector)
    plt.show()

    """Fourier basis qubits"""
    qft(circuit, image_to_binary(image).size)  # Applies QFT to encoded image
    circuit.save_statevector()
    result = simulator.run(circuit).result()
    statevector = result.get_statevector()
    plot_bloch_multivector(statevector)
    plt.show()

    # Print the circuit
    print(circuit)

    # Draw the circuit as text
    print(circuit.draw(output='text'))

In [ ]:
if __name__ == "__main__":
    main()

--2024-08-12 22:08:51--  https://github.com/ica574/quantum-image-processing/raw/main/test.png
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ica574/quantum-image-processing/main/test.png [following]
--2024-08-12 22:08:51--  https://raw.githubusercontent.com/ica574/quantum-image-processing/main/test.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [image/png]
Saving to: ‘test.png.6’

test.png.6          100%[===================>]      99  --.-KB/s    in 0s      

2024-08-12 22:08:51 (2.39 MB/s) - ‘test.png.6’ saved [99/99]

                                                                  